

# 1.  Patient Health Data Wrangling & ML Preprocessing Project



In [65]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler

READ AND CHECK DATA SHAPE DESCRIPTION NULLS

In [66]:
df =pd.read_csv('Patient_Health_Records_Raw.csv')
print('SHAPE\n',df.shape)
print('\nNULL CHECK\n',df.isnull().sum())
print('\nDuplicates\n',df.duplicated().sum())

SHAPE
 (10000, 17)

NULL CHECK
 Patient_ID            507
Name                    0
Age                     0
Gender               1707
City                 1003
BMI                  2418
Blood_Pressure       2030
Heart_Rate              0
Cholesterol_Level    2052
Diabetic             1641
Smoker               3234
Medications          4024
Last_Visit_Date         0
Follow_Up            2524
Diagnosis_Code       1675
Notes                4024
Has_Disease          2520
dtype: int64

Duplicates
 0


CATEGORICAL COLUMNS

In [67]:
#Name
df['Name']=df['Name'].str.title()
df['Name']=df['Name'].str.replace(',','')
df['Name']=df['Name'].str.strip()
empty=['','None','Nan','nan','unknown','n/a','N/A',np.nan]
df['Name']=df['Name'].replace(empty,np.nan)
df['Name']=df['Name'].fillna(df['Name'].mode()[0])
#Gender Manage
gender ={
    'male':'M',
    'female':'F',
    'm':'M',
    'f':'F',
}
df['Gender']=df['Gender'].astype(str).str.lower().str.strip()
df['Gender']=df['Gender'].replace(gender)
df['Gender']=df['Gender'].replace(['unknown','nan','Nan',''],np.nan)
df['Gender']=df['Gender'].fillna(df['Gender'].mode()[0])

#Blood Pressure

def correct_bp(x):
  x=str(x).lower()
  x=str(x).strip()
  if x in empty:
    return np.nan
  elif 'over' in x:
    x=x.replace(' over ','/')
    return x
  elif '-' in x:
    x=x.replace(' - ','/')
    return x
  else:
    return x

df['Blood_Pressure']=df['Blood_Pressure'].apply(correct_bp)
df['Blood_Pressure']=df['Blood_Pressure'].fillna(df['Blood_Pressure'].mode()[0])

#BMI COLUMN
def BMI_standard(x):
  x=str(x).lower()
  x=str(x).strip()
  if x in empty:
    return np.nan
  elif 'kg/m2' in x:
    x=x.replace('kg/m2','')
  return float(x)

df['BMI']=df['BMI'].apply(BMI_standard)
df['BMI']=df['BMI'].fillna(df['BMI'].mean())

#AGE
age = {
    'twenty':'20',
    'thirty':'30',
    'forty':'40',
    'fifty':'50'
}
df['Age']=df['Age'].replace(age)
def age_standard(x):
  x=str(x).lower()
  x=str(x).strip()
  if x in empty:
    return np.nan
  elif '-' in x:
    x=x.replace('-','')
  x=int(x)
  if x > 100:
    x=x/10
  return int(x)

df['Age']=df['Age'].apply(age_standard)
df['Age']=df['Age'].fillna(df['Age'].mean())

#city
def city_standard(x):
  x=str(x).lower()
  x=str(x).replace(' ','')
  x=str(x).strip()
  if x in empty:
    return np.nan
  city ={
      'boston':'Boston',
      'la'  :'Los Angeles',
      'newyork':'New York',
      'nweyork':'New York',
      'ny':'New York',
      'chicago':'Chicago',
  }
  city.get(x,x.title())
  return x.title()
df['City']=df['City'].apply(city_standard)
df['City']=df['City'].fillna(df['City'].mode()[0])



#cholesterol_level
def correct_cholesterol(x):
  x=str(x).lower()
  x=str(x).strip()
  if x in empty:
    return np.nan
  cholesterol = {
    'high':'250',
    'low' : '100',
    'normal':'125'
}

  return cholesterol.get(x,x)

df['Cholesterol_Level']=df['Cholesterol_Level'].apply(correct_cholesterol)
df['Cholesterol_Level']=df['Cholesterol_Level'].fillna(df['Cholesterol_Level'].mode()[0])




In [68]:
#patient_ID
df['Patient_ID']=df['Patient_ID'].astype(str)
df['Patient_ID']=df['Patient_ID'].str.replace('ID-','',regex=False)
df['Patient_ID']=df['Patient_ID'].str.strip()
df['Patient_ID']=pd.to_numeric(df['Patient_ID'],errors='coerce')
df['Patient_ID'] = df['Patient_ID'].fillna(pd.Series(df.index + 1, index=df.index))
df['Patient_ID']=df['Patient_ID'].astype(int)
df['Patient_ID']=df['Patient_ID'].astype(str)
df['Patient_ID']='ID-'+df['Patient_ID']

#AGE
def clean_age(x):
    try:
        x = str(x).strip().lower()
        if x in empty:
            return np.nan
        # convert words to numbers
        if 'twenty' in x:
            return 20
        val = float(x)
        # r
        if val < 0 or val > 120:
            return np.nan
        return val
    except:
        return np.nan

df['Age'] = df['Age'].apply(clean_age)
df['Age'] = df['Age'].fillna(df['Age'].median())
#hart rate
def clean_heart_rate(x):
    try:
        x = str(x).strip().lower()
        if x in empty or not x.replace('.', '').isdigit():
            return np.nan
        val = float(x)
        # keep realistic human range
        if val < 40 or val > 180:
            return np.nan
        return val
    except:
        return np.nan

df['Heart_Rate'] = df['Heart_Rate'].apply(clean_heart_rate)
df['Heart_Rate'] = df['Heart_Rate'].fillna(df['Heart_Rate'].median())

In [69]:
#diabetic
def clean_diabetic(x):
    x = str(x).strip().lower()
    if x in empty:
        return np.nan
    mapping = {'yes': 1, 'y': 1, 'no': 0, 'n': 0}
    return mapping.get(x, np.nan)

df['Diabetic'] = df['Diabetic'].apply(clean_diabetic)
df['Diabetic'] = df['Diabetic'].fillna(df['Diabetic'].mode()[0])



In [70]:
#smoker
def clean_smoker(x):
    x = str(x).strip().lower()
    if x in empty:
        return np.nan
    mapping = {
        'yes': 1, 'y': 1, 'no': 0, 'n': 0,
        'ex-smoker': 2, 'former': 2
    }
    return mapping.get(x, np.nan)

df['Smoker'] = df['Smoker'].apply(clean_smoker)
df['Smoker'] = df['Smoker'].fillna(df['Smoker'].mode()[0])
#clean meds
def clean_meds(x):
    x = str(x).lower().strip()
    if x in empty:
        return np.nan
    x = x.replace(';', ',')
    x = x.replace('/', ',')
    x = x.replace(' ,', ',').replace(', ', ',')
    return x

df['Medications'] = df['Medications'].apply(clean_meds)
df['Medications'] = df['Medications'].fillna('none')

df['Last_Visit_Date'] = pd.to_datetime(df['Last_Visit_Date'], errors='coerce', infer_datetime_format=True)
df['Last_Visit_Date'] = df['Last_Visit_Date'].fillna(df['Last_Visit_Date'].mode()[0])


#Follow_Up

def clean_follow_up(x):
    x = str(x).strip().lower()
    if x in empty or x == 'n/a':
        return np.nan
    try:
        return int(float(x))
    except:
        return np.nan

df['Follow_Up'] = df['Follow_Up'].apply(clean_follow_up)
df['Follow_Up'] = df['Follow_Up'].fillna(df['Follow_Up'].mode()[0])


# Diagnosis_Code

df['Diagnosis_Code'] = df['Diagnosis_Code'].replace(empty, np.nan)
df['Diagnosis_Code'] = df['Diagnosis_Code'].fillna('Unknown')

#notes
df['Notes'] = df['Notes'].astype(str).str.strip()
df['Notes'] = df['Notes'].replace(empty, np.nan)
df['Notes'] = df['Notes'].fillna('No Notes')


#Has_Disease (Target Column)

def clean_target(x):
    x = str(x).strip().lower()
    if x in ['1', 'yes', 'y']:
        return 1
    elif x in ['0', 'no', 'n']:
        return 0
    else:
        return np.nan

df['Has_Disease'] = df['Has_Disease'].apply(clean_target)
df['Has_Disease'] = df['Has_Disease'].fillna(df['Has_Disease'].mode()[0])


/tmp/ipython-input-3961917933.py:27: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Last_Visit_Date'] = pd.to_datetime(df['Last_Visit_Date'], errors='coerce', infer_datetime_format=True)


Machine learning pipe Normalization + encoding

In [71]:
binary_map = {
    'Gender': {'M': 1, 'F': 0, 'male': 1, 'female': 0},
    'Smoker': {'yes': 1, 'no': 0, 'former': 2, 'ex-smoker': 2},
    'Diabetic': {'yes': 1, 'no': 0, 'y': 1, 'n': 0}
}

for col, mapping in binary_map.items():
    if col in df.columns:
        df[col] = df[col].astype(str).str.lower().map(mapping).fillna(0)

# -label encode
label_cols = ['City', 'Diagnosis_Code', 'Medications', 'Has_Disease']

le = LabelEncoder()
for col in label_cols:
    if col in df.columns:
        df[col] = df[col].astype(str)
        df[col] = le.fit_transform(df[col])


# NORMALIZATION (NUMERIC FEATURES)


# List numeric columns to scale (exclude already encoded ones)
numeric_cols = [
    'Age', 'BMI', 'Heart_Rate', 'Cholesterol_Level',
    'Follow_Up', 'Blood_Pressure'
]

# Convert to numeric and handle NaNs
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].fillna(df[col].mean())

# Min-Max Scaling (range 0–1)
scaler = MinMaxScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
#check

print("\n Data encoding & normalization complete!\n")
print(df.info())
print("\nSample rows:\n", df.sample(10))


 Data encoding & normalization complete!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Patient_ID         10000 non-null  object        
 1   Name               10000 non-null  object        
 2   Age                10000 non-null  float64       
 3   Gender             10000 non-null  float64       
 4   City               10000 non-null  int64         
 5   BMI                10000 non-null  float64       
 6   Blood_Pressure     0 non-null      float64       
 7   Heart_Rate         10000 non-null  float64       
 8   Cholesterol_Level  10000 non-null  float64       
 9   Diabetic           10000 non-null  float64       
 10  Smoker             10000 non-null  float64       
 11  Medications        10000 non-null  int64         
 12  Last_Visit_Date    10000 non-null  datetime64[ns]
 13  Follow_Up          

/usr/local/lib/python3.12/dist-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/usr/local/lib/python3.12/dist-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [72]:
df.to_csv('Clean_Heath_Records.csv')